In [1]:
!pip install fastapi uvicorn pandas requests nest-asyncio pyngrok


   ---------------------------------------- 0.0/95.6 kB ? eta -:--:--
   ---------------------------------------- 95.6/95.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/66.4 kB ? eta -:--:--
   ---------------------------------------- 66.4/66.4 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/73.0 kB ? eta -:--:--
   ---------------------------------------- 73.0/73.0 kB 4.2 MB/s eta 0:00:00


In [13]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load TOMTOM_API_KEY from .env file
load_dotenv()
API_KEY = os.getenv("TOMTOM_API_KEY")
assert API_KEY, "No TOMTOM_API_KEY found. Put it in a .env file first."

# Checking a few well-known NYC locations
NYC_POINTS = [
    {"name": "Times Square",        "lat": 40.7580, "lng": -73.9855},
    {"name": "Central Park South",  "lat": 40.7651, "lng": -73.9769},
    {"name": "Grand Central",       "lat": 40.7527, "lng": -73.9772},
    {"name": "Wall Street",         "lat": 40.7066, "lng": -74.0090},
    {"name": "Brooklyn Bridge",     "lat": 40.7061, "lng": -73.9969},
]


In [15]:
def get_flow(lat, lng):
    url = (
        "https://api.tomtom.com/traffic/services/4/"
        f"flowSegmentData/relative0/10/json?point={lat},{lng}&key={API_KEY}"
    )
    r = requests.get(url)
    return r.json().get("flowSegmentData")


In [17]:
test = get_flow(40.7580, -73.9855)  # Times Square
print(test)


{'frc': 'FRC4', 'currentSpeed': 11, 'freeFlowSpeed': 11, 'currentTravelTime': 130, 'freeFlowTravelTime': 130, 'confidence': 1, 'roadClosure': False, 'coordinates': {'coordinate': [{'latitude': 40.76110531505816, 'longitude': -73.98325866413843}, {'latitude': 40.76047227561436, 'longitude': -73.98369452310348}, {'latitude': 40.759841960167215, 'longitude': -73.98416390968119}, {'latitude': 40.7594932778618, 'longitude': -73.98440262628357}, {'latitude': 40.75920090900551, 'longitude': -73.98461049748227}, {'latitude': 40.7585840414191, 'longitude': -73.98507854295548}, {'latitude': 40.75844322008275, 'longitude': -73.98517376137552}, {'latitude': 40.75796443800581, 'longitude': -73.9855318362791}]}, '@version': '4'}


In [33]:
rows = []
for name, lat, lng in POINTS:
    d = get_flow(lat, lng)
    rows.append({
        "name": name,
        "lat": lat,
        "lng": lng,
        "currentSpeed": d.get("currentSpeed"),
        "freeFlowSpeed": d.get("freeFlowSpeed"),
        "currentTravelTime": d.get("currentTravelTime"),
        "freeFlowTravelTime": d.get("freeFlowTravelTime"),
        "confidence": d.get("confidence"),
    })

df = pd.DataFrame(rows)
df


,name,lat,lng,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence
0,Times Square,40.7580,-73.9855,11,11,130,130,1
1,Wall Street,40.7066,-74.0090,14,17,231,190,1


In [23]:
df["congestion_ratio"] = df["currentSpeed"] / df["freeFlowSpeed"]
df


,name,currentSpeed,freeFlowSpeed,congestion_ratio
0,Times Square,11,11,1.000000
1,Central Park South,11,11,1.000000
2,Grand Central,13,13,1.000000
3,Wall Street,14,17,0.823529
4,Brooklyn Bridge,44,44,1.000000


In [25]:
!pip install folium


In [39]:
import folium

# start the map centered in NYC
m = folium.Map(location=[40.733, -73.995], zoom_start=12)

# add each point from your DataFrame
for _, r in df.iterrows():
    folium.Marker(
        location=[r["lat"], r["lng"]],
        popup=f"{r['name']}: {r['currentSpeed']} mph",
    ).add_to(m)

m
